<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 817ms/step - loss: 0.6683 - accuracy: 0.5864 - val_loss: 0.7166 - val_accuracy: 0.4341
Epoch 2/10
26/26 [==============================] - 20s 780ms/step - loss: 0.6326 - accuracy: 0.6934 - val_loss: 0.6055 - val_accuracy: 0.7610
Epoch 3/10
26/26 [==============================] - 20s 762ms/step - loss: 0.5742 - accuracy: 0.7263 - val_loss: 0.5775 - val_accuracy: 0.7220
Epoch 4/10
26/26 [==============================] - 20s 769ms/step - loss: 0.5515 - accuracy: 0.7336 - val_loss: 0.4904 - val_accuracy: 0.8146
Epoch 5/10
26/26 [==============================] - 20s 769ms/step - loss: 0.4946 - accuracy: 0.7822 - val_loss: 0.4374 - val_accuracy: 0.8341
Epoch 6/10
26/26 [==============================] - 20s 765ms/step - loss: 0.4261 - accuracy: 0.8200 - val_loss: 0.3776 - val_accuracy: 0.8390
Epoch 7/10
26/26 [==============================] - 20s 758ms/step - loss: 0.3575 - accuracy: 0.8662 - val_loss: 0.3249 - val_accuracy: 0.8927

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.68.h5
26/26 - 20s - loss: 0.6780 - accuracy: 0.5706 - val_loss: 0.6798 - val_accuracy: 0.5024
Epoch 2/5

Epoch 00002: saving model to weights.02-0.69.h5
26/26 - 20s - loss: 0.6544 - accuracy: 0.6168 - val_loss: 0.6932 - val_accuracy: 0.4829
Epoch 3/5

Epoch 00003: saving model to weights.03-0.70.h5
26/26 - 20s - loss: 0.6230 - accuracy: 0.6557 - val_loss: 0.6957 - val_accuracy: 0.5073
Epoch 4/5

Epoch 00004: saving model to weights.04-0.57.h5
26/26 - 20s - loss: 0.6042 - accuracy: 0.6861 - val_loss: 0.5677 - val_accuracy: 0.7659
Epoch 5/5

Epoch 00005: saving model to weights.05-0.77.h5
26/26 - 20s - loss: 0.5731 - accuracy: 0.7165 - val_loss: 0.7727 - val_accuracy: 0.5854


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6787 - accuracy: 0.5304 - val_loss: 0.6521 - val_accuracy: 0.6683


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 21s - loss: 0.6545 - accuracy: 0.5779 - val_loss: 0.6858 - val_accuracy: 0.4634
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 21s - loss: 0.6084 - accuracy: 0.6752 - val_loss: 0.5656 - val_accuracy: 0.6634


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 21s - loss: 0.6572 - accuracy: 0.6411 - val_loss: 0.6369 - val_accuracy: 0.6732
Epoch 2/50
26/26 - 20s - loss: 0.6026 - accuracy: 0.6667 - val_loss: 0.5703 - val_accuracy: 0.7805
Epoch 3/50
26/26 - 20s - loss: 0.5342 - accuracy: 0.7506 - val_loss: 0.5922 - val_accuracy: 0.6585
Epoch 4/50
26/26 - 20s - loss: 0.4793 - accuracy: 0.7895 - val_loss: 0.4611 - val_accuracy: 0.8244
Epoch 5/50
26/26 - 20s - loss: 0.4130 - accuracy: 0.8260 - val_loss: 0.8140 - val_accuracy: 0.5122
Epoch 6/50
26/26 - 20s - loss: 0.3635 - accuracy: 0.8625 - val_loss: 0.3014 - val_accuracy: 0.9122
Epoch 7/50
26/26 - 20s - loss: 0.3145 - accuracy: 0.8808 - val_loss: 0.2763 - val_accuracy: 0.9122
Epoch 8/50
26/26 - 20s - loss: 0.2377 - accuracy: 0.9258 - val_loss: 0.1918 - val_accuracy: 0.9512
Epoch 9/50
26/26 - 20s - loss: 0.1874 - accuracy: 0.9562 - val_loss: 0.1816 - val_accuracy: 0.9415
Epoch 10/50
26/26 - 20s - loss: 0.1964 - accuracy: 0.9477 - val_loss: 0.1175 - val_accuracy: 0.9854
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 794ms/step - loss: 0.6635 - accuracy: 0.6314 - val_loss: 0.7629 - val_accuracy: 0.4341
Epoch 2/5
26/26 [==============================] - 20s 769ms/step - loss: 0.6207 - accuracy: 0.6837 - val_loss: 0.5730 - val_accuracy: 0.8049
Epoch 3/5
26/26 [==============================] - 20s 773ms/step - loss: 0.5479 - accuracy: 0.7567 - val_loss: 0.4984 - val_accuracy: 0.8244
Epoch 4/5
26/26 [==============================] - 20s 765ms/step - loss: 0.4871 - accuracy: 0.7859 - val_loss: 0.4349 - val_accuracy: 0.8537
Epoch 5/5
26/26 [==============================] - 20s 773ms/step - loss: 0.3957 - accuracy: 0.8321 - val_loss: 0.3358 - val_accuracy: 0.8780


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.631387,0.664820,0.434146,0.762854
1,1,0.683698,0.621352,0.804878,0.572997
2,2,0.756691,0.548339,0.824390,0.498368
3,3,0.785888,0.487160,0.853659,0.434921
4,4,0.832117,0.395930,0.878049,0.335759


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 29s 1s/step - loss: 0.6698 - accuracy: 0.5742 - val_loss: 0.6509 - val_accuracy: 0.7463

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 784ms/step - loss: 0.6370 - accuracy: 0.6727 - val_loss: 0.6452 - val_accuracy: 0.5707

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 761ms/step - loss: 0.6202 - accuracy: 0.7324 - val_loss: 0.6385 - val_accuracy: 0.5951

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 773ms/step - loss: 0.6120 - accuracy: 0.7153 - val_loss: 0.6255 - val_accuracy: 0.7366

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 765ms/step - loss: 0.6073

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 23s 895ms/step - loss: 0.6599 - accuracy: 0.6290 - val_loss: 0.6309 - val_accuracy: 0.6439
Epoch 2/50
26/26 [==============================] - 20s 776ms/step - loss: 0.6017 - accuracy: 0.7165 - val_loss: 0.5374 - val_accuracy: 0.8488
Epoch 3/50
26/26 [==============================] - 20s 784ms/step - loss: 0.5326 - accuracy: 0.7579 - val_loss: 0.4521 - val_accuracy: 0.8585
Epoch 4/50
26/26 [==============================] - 20s 761ms/step - loss: 0.4426 - accuracy: 0.8297 - val_loss: 0.3664 - val_accuracy: 0.8927
Epoch 5/50
26/26 [==============================] - 20s 754ms/step - loss: 0.3636 - accuracy: 0.8577 - val_loss: 0.3121 - val_accuracy: 0.9122
Epoch 6/50
26/26 [==============================] - 20s 761ms/step - loss: 0.2781 - accuracy: 0.9124 - val_loss: 0.2092 - val_accuracy: 0.9561
Epoch 7/50
25/26 [===========================>..] - ETA: 0s - loss: 0.2403 - accuracy: 0.9225
Epoch 00007: ReduceLROnPlateau reducing learning

In [ ]:
%tensorboard --logdir log_dir